In [1]:
import tensorflow as tf
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments
from datasets import Dataset
import sentencepiece as spm
import argparse
import pandas as pd
import re
import transformers
import numpy as np
import evaluate

print(transformers.__version__)

4.27.1


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
train_data = pd.read_table("./ratings_train.txt")
test_data = pd.read_table("./ratings_test.txt")

In [4]:
train_data = train_data.dropna(axis=0)
test_data= test_data.dropna(axis=0)

train_sentences = train_data["document"]
train_label = list(train_data["label"])

test_sentences = test_data["document"]
test_label = list(test_data["label"])

In [5]:
def preprocess(sentence):
    sentence = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s.,?!\'\"]", "", sentence)
    sentence = re.sub("ㅋ+", "ㅋㅋ", sentence)
    sentence = re.sub("ㅎ+", "ㅎㅎ", sentence)
    sentence = re.sub("ㅜ+", "ㅜ", sentence)
    sentence = re.sub("ㅠ+", "ㅠ", sentence)
    sentence = re.sub("\"+", "\"", sentence)
    sentence = re.sub(r"\.+", "..", sentence)
    sentence = re.sub("!+", "!", sentence)
    sentence = re.sub("\?+", "?", sentence)
    sentence.lower().strip()
    
    return sentence

In [6]:
train_corpus = []
train_labels = []
for i, sentence in enumerate(train_sentences):
    if len(sentence) < 60:
        train_corpus.append(preprocess(sentence))
        train_labels.append(train_label[i])

In [7]:
test_corpus = []
test_labels = []
for i, sentence in enumerate(test_sentences):
    if len(sentence) < 60:
        test_corpus.append(preprocess(sentence))
        test_labels.append(test_label[i])

In [8]:
train_data = pd.DataFrame([i for i in zip(train_corpus, train_labels)], columns = ["text", "labels"])
test_data = pd.DataFrame([i for i in zip(test_corpus, test_labels)], columns = ["text", "labels"])

In [9]:
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

In [10]:
train_dataset = train_dataset.class_encode_column("labels")
test_dataset = test_dataset.class_encode_column("labels")

Stringifying the column:   0%|          | 0/128937 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/128937 [00:00<?, ? examples/s]

Stringifying the column:   0%|          | 0/42881 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/42881 [00:00<?, ? examples/s]

In [11]:
model =  AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

In [12]:
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding=True, max_length=60)

In [13]:
train_datasets = train_dataset.map(tokenize_function, batched=True)
test_datasets = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/128937 [00:00<?, ? examples/s]

Map:   0%|          | 0/42881 [00:00<?, ? examples/s]

In [14]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [15]:
training_args = TrainingArguments(
    output_dir="./nsmc",
    learning_rate=2e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

In [16]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_datasets,
    eval_dataset=test_datasets,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [17]:
trainer.train()

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.274200,0.265766,0.888832
2,0.214500,0.261155,0.897554
3,0.165600,0.276434,0.895478
4,0.129900,0.301314,0.896691
5,0.101900,0.328449,0.895875


TrainOutput(global_step=5040, training_loss=0.18322108359563918, metrics={'train_runtime': 6438.0918, 'train_samples_per_second': 100.136, 'train_steps_per_second': 0.783, 'total_flos': 1.813851335103972e+16, 'train_loss': 0.18322108359563918, 'epoch': 5.0})

In [ ]:
train_datasets